In [32]:
with open('./sample_data/sample.txt', 'r') as f:
  docs = f.readlines()
#for id, doc in enumerate(docs):
  #print('[{}] : {}...'.format(id, doc[:30]))

from sklearn.feature_extraction.text import TfidfVectorizer

#tfidf 벡터 매트릭스 생성
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(docs)
#print('type of tfidf_matrix {}'.format(type(tfidf_matrix)))
#print('shape of tfidf_matrix {}'.format(tfidf_matrix.shape))

vocab = sorted(tfidf.vocabulary_.items())
#vocab[10:20]

import pandas as pd

df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf.get_feature_names_out())
#print(df.head(5))

tfidf_table = tfidf_matrix.toarray()

keywords = []
for weight in tfidf_table:
  w_vec = list(enumerate(weight))
  w_vec = sorted(w_vec, key=lambda x : x[1], reverse=True)
  #print(w_vec[:3])
  keywords.append(w_vec)

import numpy as np
def tfidf_rank(tfidf_matrix):
  rank = []
  avg, stddev = 0.0, 0.0
  #문서 별 tfidf 가중치의 합 계산 : (문서id, 가중치 합)
  for idx, tfidf in enumerate(tfidf_matrix):
    rank.append((idx, tfidf.sum()))
    
  #가중치의 합이 높은 문서 순으로 정렬
  rank.sort(key=lambda x : x[1], reverse=True)
  #tfidf의 평균과 표준편차 계산
  tfidf_sum = [tfidf.sum() for tfidf in tfidf_matrix]
  avg = np.mean(tfidf_sum)
  stddev = np.std(tfidf_sum)
  return rank, avg, stddev

rank, avg, stddev = tfidf_rank(tfidf_matrix)

#print(rank[:2])
#print('avg = {}, stddev = {}'.format(avg, stddev))

df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf.get_feature_names_out())
result = df.sum()
result = result.sort_values(ascending=False)
#print(result[:10])

tfidf2 = TfidfVectorizer(stop_words='english')
tfidf_matrix2 = tfidf2.fit_transform(docs)
#print('shape of tfidf_matrix2 = {}'.format(tfidf_matrix2.shape))
df = pd.DataFrame(tfidf_matrix2.toarray(), columns=tfidf2.get_feature_names_out())
result = df.sum()
result = result.sort_values(ascending=False)
#print(result[:10])

#과제 2번
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel

from functools import reduce

cos_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

rank5 = {0:[0,0]}
for i, data in enumerate(cos_sim):
  for j, doc in enumerate(data):
    seconddata = doc
    sortkeys = sorted(rank5.keys())
    #print(sortkeys,len(sortkeys))
    if doc < 0.9:
      if min(rank5.keys())<doc:
        rank5[doc] = [i,j]
      if len(rank5.keys()) > 5:
        del rank5[sortkeys[0]]
#print(rank5)
rank5 = dict(sorted(rank5.items(), reverse=True))
cnt = 1
for key,value in rank5.items():
  print("{0:2d}번 문서와 {1:2d}번 문서의 코사인 유사도가 {2:0.02f}%로 {3}번째로 유사합니다.".format(value[0],value[1],key*100,cnt))
  cnt += 1

10번 문서와  8번 문서의 코사인 유사도가 33.73%로 1번째로 유사합니다.
 9번 문서와  8번 문서의 코사인 유사도가 30.11%로 2번째로 유사합니다.
13번 문서와  8번 문서의 코사인 유사도가 27.15%로 3번째로 유사합니다.
 6번 문서와  5번 문서의 코사인 유사도가 24.88%로 4번째로 유사합니다.
 9번 문서와 10번 문서의 코사인 유사도가 24.81%로 5번째로 유사합니다.
